# Using QPU selectors directly

##### First, let's take a glance at all of the backends available to us via Qiskit Runtime. We will ignore all simulators.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

for backend in service.backends():
    if backend.name.split("_")[0] == "simulator" or backend.simulator:
        continue
    print(
        f"Backend name: {backend.name}    ",
        f"Status: {backend.status().status_msg}    ",
        f"Operational: {backend.status().operational}    ",
        f"Queue size: {backend.status().pending_jobs}    ",
        f"# qubits: {backend.num_qubits}",
    )

Backend name: ibmq_mumbai     Status: active     Operational: True     Queue size: 531     # qubits: 27
Backend name: ibmq_jakarta     Status: active     Operational: True     Queue size: 0     # qubits: 7
Backend name: ibm_cusco     Status: active     Operational: True     Queue size: 176     # qubits: 127
Backend name: ibm_algiers     Status: active     Operational: True     Queue size: 2     # qubits: 27
Backend name: ibmq_manila     Status: active     Operational: True     Queue size: 0     # qubits: 5
Backend name: ibm_lagos     Status: dedicated     Operational: True     Queue size: 41     # qubits: 7
Backend name: ibm_auckland     Status: active     Operational: True     Queue size: 145     # qubits: 27
Backend name: ibm_perth     Status: active     Operational: True     Queue size: 498     # qubits: 7
Backend name: ibmq_guadalupe     Status: active     Operational: True     Queue size: 1     # qubits: 16
Backend name: ibm_cairo     Status: active     Operational: True     Queue

##### Use `IBMLeastBusyQPUSelector` to select the backend with at least 20 qubits that has the shortest queue.

In [2]:
from quantum_serverless_tools.selectors import IBMLeastBusyQPUSelector

IBMLeastBusyQPUSelector(service).get_backend(min_num_qubits=20)

<IBMBackend('ibm_hanoi')>

##### Use `IBMLeastNoisyQPUSelector` to select the backend with at least 40 qubits which yields the least-noisy qubit subgraph for a given circuit.

In [3]:
from qiskit import QuantumCircuit
from quantum_serverless_tools.selectors import IBMLeastNoisyQPUSelector

qc = QuantumCircuit(40)
qc.h(0)
for n in range(qc.num_qubits - 1):
    qc.cx(n, n + 1)

IBMLeastNoisyQPUSelector(service=service, circuit=qc).get_backend(min_num_qubits=40)

<IBMBackend('ibm_cusco')>